In [9]:
import requests
import os
import re
from astropy import units as u
from astropy.coordinates import Angle
from astropy.coordinates import SkyCoord

import pandas as pd

WDS_FORMAT_DSC = """
    COLUMN     Format                     DATA
    --------   ------         ----------------------------
    1  -  10   A10             2000 Coordinates
    11 -  17   A7              Discoverer & Number
    18 -  22   A5              Components
    24 -  27   I4              Date (first)
    29 -  32   I4              Date (last)
    34 -  37   I4              Number of Observations (up to 9999)
    39 -  41   I3              Position Angle (first - XXX)
    43 -  45   I3              Position Angle (last  - XXX)
    47 -  51   F5.1            Separation (first)
    53 -  57   F5.1            Separation (last)
    59 -  63   F5.2            Magnitude of First Component
    65 -  69   F5.2            Magnitude of Second Component
    71 -  79   A9              Spectral Type (Primary/Secondary)
    81 -  84   I4              Primary Proper Motion (RA)
    85 -  88   I4              Primary Proper Motion (Dec)
    90 -  93   I4              Secondary Proper Motion (RA)
    94 -  97   I4              Secondary Proper Motion (Dec)
    99 - 106   A8              Durchmusterung Number
   108 - 111   A4              Notes
   113 - 130   A18             2000 arcsecond coordinates
"""

In [10]:
lines = WDS_FORMAT_DSC.splitlines()
lines = lines[3:]
lines = [x.strip() for x in lines]
print(lines)
wds_format = []
for line in lines:
    col_start, _ , col_end, fmt, dt = re.split(r'\s+', line, 4)
    print(col_start, col_end, fmt, dt)
    wds_format.append((col_start, col_end, fmt, dt))
print(wds_format)

['1  -  10   A10             2000 Coordinates', '11 -  17   A7              Discoverer & Number', '18 -  22   A5              Components', '24 -  27   I4              Date (first)', '29 -  32   I4              Date (last)', '34 -  37   I4              Number of Observations (up to 9999)', '39 -  41   I3              Position Angle (first - XXX)', '43 -  45   I3              Position Angle (last  - XXX)', '47 -  51   F5.1            Separation (first)', '53 -  57   F5.1            Separation (last)', '59 -  63   F5.2            Magnitude of First Component', '65 -  69   F5.2            Magnitude of Second Component', '71 -  79   A9              Spectral Type (Primary/Secondary)', '81 -  84   I4              Primary Proper Motion (RA)', '85 -  88   I4              Primary Proper Motion (Dec)', '90 -  93   I4              Secondary Proper Motion (RA)', '94 -  97   I4              Secondary Proper Motion (Dec)', '99 - 106   A8              Durchmusterung Number', '108 - 111   A4           

In [11]:
wds_data_file = "C:\\Users\eligo\Downloads\\wdsweb_summ2.txt"

wds_data = None
with open(wds_data_file,'r') as f:
    wds_data = f.readlines()
    
print(wds_data[0], wds_data[-1])

00000+7530A  1248      1904 1982    5 246 235   0.8   0.6 10.27 11.5  A7IV      +034+005          +74 1056      000006.64+752859.8
 23599-3112TDT4315      1991 1991    1 248 248   0.7   0.7 11.54 11.90           +001+000 +001+000 -3119543      235953.60-311201.2


In [12]:
wds_data_pd = []

def parse_wds_data(wds_fmt, wds_data_line):
    wds_pr = []
    for wds_fmt_e in wds_fmt:
        col_sr = int(wds_fmt_e[0]) - 1
        col_sp = int(wds_fmt_e[1])
        wds_pr.append(wds_data_line[col_sr:col_sp])
    return tuple(wds_pr)

for wds_data_line in wds_data:
    wds_data_pd.append(parse_wds_data(wds_format, wds_data_line))

print(wds_data_pd[0])
print(wds_data_pd[-1])

('00000+7530', 'A  1248', '     ', '1904', '1982', '   5', '246', '235', '  0.8', '  0.6', '10.27', '11.5 ', 'A7IV     ', '+034', '+005', '    ', '    ', '+74 1056', '    ', '000006.64+752859.8')
('23599-3112', 'TDT4315', '     ', '1991', '1991', '   1', '248', '248', '  0.7', '  0.7', '11.54', '11.90', '         ', '+001', '+000', '+001', '+000', '-3119543', '    ', '235953.60-311201.2')


In [13]:
ann_data = []
for wds_data_e in wds_data_pd:
    desc = wds_data_e[1] + wds_data_e[2]
    desc = desc.strip()
    
    try:
        a, sep, b = re.split(r'([+-])', wds_data_e[-1], 2)
        _, ra_h, _, ra_m, ra_s = re.split(r'(\d{2})', a, 2)
        ra = f"{ra_h}h{ra_m}m{ra_s}s"
        
        _, dec_d, _, dec_m, dec_s = re.split(r'(\d{2})', b, 2)
        dec = f"{dec_d}d{dec_m}m{dec_s}s"
        dec = sep + dec
        
        #ra = Angle(ra).degree
        #dec = Angle(dec).degree
        c = SkyCoord(ra, dec, frame="icrs", equinox="J2000")
        ann_data.append((desc, c.ra.degree, c.dec.degree))
    except Exception as e:
        print("failed to process the following entry")
        print(wds_data_e)
        print(e)
        continue
    

wds_df = pd.DataFrame(ann_data, columns=['NAME', 'RA', 'DEC'])
print(wds_df.info())

failed to process the following entry
('00123-2729', 'LDS2101', '     ', '1954', '2017', '   2', ' 64', ' 57', '  5.0', '  1.9', '16.0 ', '19.8 ', '         ', '+085', '+027', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('00172+0332', 'LDS3014', '     ', '1951', '1951', '   1', '294', '294', '  1.5', '  1.5', '18.3 ', '20.2 ', '         ', '-070', '-121', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('00198-2334', 'LDS2116', '     ', '1954', '1954', '   1', ' 34', ' 34', ' 37.0', ' 37.0', '15.7 ', '20.0 ', '         ', '+056', '-047', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('00316+1344', 'LDS9091', '     ', '1961', '1961', '   1', '180', '180', '  3.0', '  3.0', '15.9 ', '16.5 ', '      

failed to process the following entry
('03005+4916', 'ALT  13', '     ', '  -1', '  -1', '   0', '136', '136', '  3.1', '  3.1', '18.4 ', '18.6 ', '         ', '    ', '    ', '    ', '    ', '        ', '    ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('03005+4911', 'ALT   8', '     ', '  -1', '  -1', '   0', '277', '277', '  2.8', '  2.8', '13.4 ', '17.4 ', '         ', '    ', '    ', '    ', '    ', '        ', '    ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('03005+4909', 'ALT  16', '     ', '  -1', '  -1', '   0', '359', '359', '  3.2', '  3.2', '12.9 ', '17.9 ', '         ', '    ', '    ', '    ', '    ', '        ', '    ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('03011+4849', 'ALT  15', '     ', '  -1', '  -1', '   0', '348', '348', '  1.3', '  1.3', '15.4 ', '18.4 ', '      

failed to process the following entry
('05379-0452', 'BU 1051', '     ', '1889', '1933', '   3', ' 25', ' 25', '  0.8', '  0.9', '10.1 ', '10.7 ', '         ', '    ', '    ', '    ', '    ', '        ', '    ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('05406+2632', 'ITF  45', '     ', '1951', '1997', '   2', ' 63', ' 63', '  4.3', '  4.0', '13.5 ', '14.5 ', 'K1V+K6V  ', '+025', '+010', '+027', '+008', '        ', ' V  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('05438+8304', 'LDS1614', '     ', '1963', '1963', '   1', ' 16', ' 16', '  2.5', '  2.5', '17.7 ', '20.4 ', '         ', '-587', '+036', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('05473-1011', 'RST3432', '     ', '1938', '1943', '   2', '251', '250', '  0.9', '  1.0', '10.5 ', '12.8 ', '      

failed to process the following entry
('07268-0822', 'OL  169', '     ', '1937', '1937', '   2', ' 21', ' 21', '  2.0', '  2.0', ' 9.8 ', '12.  ', '         ', '    ', '    ', '    ', '    ', '        ', '    ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('07283-1353', 'BRT1879', '     ', '1902', '1902', '   1', '126', '126', '  4.8', '  4.8', '11.4 ', '13.1 ', '         ', '+000', '-012', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('07312-0343', 'J  1065', 'BC   ', '2015', '2015', '   1', '151', '151', '  5.1', '  5.1', '11.67', '15.0 ', '         ', '+025', '-084', '    ', '    ', '        ', '    ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('07404+0603', 'HEI 850', '     ', '1994', '1994', '   1', ' 21', ' 21', '  0.3', '  0.3', '10.9 ', '10.9 ', '      

failed to process the following entry
('11310+6124', 'LDS2603', '     ', '1965', '1965', '   1', '268', '268', ' 10.0', ' 10.0', '21.  ', '21.  ', '         ', '-127', '+005', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('11319+6924', 'LDS1735', '     ', '1966', '1966', '   1', '115', '115', '  1.2', '  1.2', '14.3 ', '14.8 ', '         ', '-230', '+001', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('11406+0454', 'HJ 1193', '     ', '1828', '1828', '   1', '100', '100', ' 13.0', ' 13.0', ' 9.  ', '11.  ', '         ', '    ', '    ', '    ', '    ', '        ', '    ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('12189+3119', 'LDS1291', '     ', '1963', '2018', '   2', '305', '322', ' 39.0', ' 53.9', '14.4 ', '21.0 ', '      

failed to process the following entry
('15031-4200', 'B  1257', 'BC   ', '2019', '2019', '   1', '306', '306', '  0.1', '  0.1', '12.8 ', '13.6 ', '         ', '+110', '-203', '    ', '    ', '        ', '   R', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('15121+1824', 'LDS5163', '     ', '1960', '1960', '   1', ' 91', ' 91', '172.0', '172.0', '16.8 ', '19.0 ', '         ', '-041', '-102', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('15224-4301', 'LDS 525', '     ', '1920', '1920', '   1', ' 90', ' 90', ' 11.0', ' 11.0', '13.8 ', '15.2 ', '         ', '+031', '-051', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('15230-3052', 'YMG  71', 'BC   ', '2019', '2019', '   1', '330', '330', '  0.4', '  0.4', '14.5 ', '15.6 ', '      

failed to process the following entry
('17132-1834', 'ELP  39', 'BC   ', '2018', '2018', '   1', '102', '102', '  0.4', '  0.4', ' 6.5 ', ' 7.32', 'M3Ve     ', '    ', '    ', '    ', '    ', '        ', '   R', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('17154+6252', 'LDS1441', '     ', '1963', '1963', '   1', '298', '298', '194.0', '194.0', '16.9 ', '20.6 ', '         ', '-021', '-109', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('17260-2727', 'LDS 596', '     ', '1920', '2015', '   2', '315', '312', ' 19.0', ' 19.6', '14.8 ', '16.0 ', '         ', '-039', '-010', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('17442-2727', 'SZA   3', 'Ca,Cb', '2018', '2019', '   6', '135', '143', '  1.0', '  0.3', '11.5 ', '11.5 ', '      

failed to process the following entry
('19399+3037', 'BRT3347', 'AB   ', '1900', '1900', '   1', '211', '211', '  1.7', '  1.7', '10.3 ', '11.5 ', '         ', '    ', '    ', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('19399+3037', 'BRT3347', 'AC   ', '1900', '1900', '   1', ' 74', ' 74', '  2.2', '  2.2', '10.3 ', '11.4 ', '         ', '    ', '    ', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('19421+2934', 'OL  121', '     ', '1924', '1924', '   1', '221', '221', '  0.9', '  0.9', '10.  ', '10.5 ', '         ', '    ', '    ', '    ', '    ', '        ', '    ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('19432+0448', 'J  1290', '     ', '1916', '1916', '   1', '214', '214', '  2.5', '  2.5', '10.0 ', '10.3 ', '      

failed to process the following entry
('21092+4152', 'LDS2491', '     ', '1954', '1954', '   1', '184', '184', ' 17.0', ' 17.0', '16.8 ', '21.3 ', '         ', '+007', '+029', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('21121+4123', 'LDS2494', '     ', '1954', '1954', '   1', '212', '212', '  2.0', '  2.0', '15.5 ', '16.5 ', '         ', '+082', '+050', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('21135+2512', 'POU5269', '     ', '1898', '2003', '   2', '314', '270', '  2.3', '  4.5', '11.4 ', '11.6 ', '         ', '    ', '    ', '    ', '    ', '        ', '    ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('21135+1604', 'SMA 143', '     ', '1921', '1921', '   1', '  8', '  8', '  8.7', '  8.7', '12.0 ', '12.5 ', '      

failed to process the following entry
('23267+6124', 'MLR 514', '     ', '1974', '1974', '   1', ' 15', ' 15', '  1.4', '  1.4', ' 9.7 ', '11.5 ', '         ', '    ', '    ', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('23316-0838', 'LDS6402', '     ', '1958', '1958', '   1', '138', '138', '  2.5', '  2.5', '17.6 ', '17.9 ', '         ', '    ', '    ', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('23376-2333', 'LDS6406', '     ', '  -1', '  -1', '   0', '194', '194', '  8.0', '  8.0', '17.5 ', '18.6 ', '         ', '    ', '    ', '    ', '    ', '        ', ' X  ', '      .         . ')
not enough values to unpack (expected 3, got 1)
failed to process the following entry
('23402+2959', 'MLB 672', '     ', '1930', '1930', '   1', '111', '111', '  4.1', '  4.1', ' 9.9 ', '12.3 ', '      

In [14]:
output_csv = "C:\\Users\eligo\Downloads\\wds_ann.txt"
wds_df.to_csv(output_csv, sep ='\t', index=False)